<a href="https://colab.research.google.com/github/muskan0004/CloudThreatDetector/blob/main/cloudthreatdectector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [29]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# File path to the uploaded CSV in Google Colab
file_path = "/content/sample_data/cloudlogs.csv"

def preprocess_data(file_path):
    logs = pd.read_csv(file_path)

    # Print raw data to verify it's loaded correctly
    print("Raw Data:\n", logs.head())

    # Convert timestamp column to datetime
    logs['timestamp'] = pd.to_datetime(logs['timestamp'])

    # Encode categorical data
    enc = LabelEncoder()
    logs['user_id'] = enc.fit_transform(logs['user_id'])
    logs['event_type'] = enc.fit_transform(logs['event_type'])

    # Feature engineering
    logs['failed_login_ratio'] = logs['failed_logins'] / (logs['successful_logins'] + 1)

    # Define features and labels
    X = logs.drop(columns=['timestamp', 'ip_address', 'label'])  # Features
    y = logs['label']  # Target variable

    return X, y  # Return processed data

# Run the function and store results
X, y = preprocess_data(file_path)

# Display the first few rows of the processed data
print("\nProcessed Features:\n", X.head())

Raw Data:
              timestamp  user_id   ip_address     event_type  data_transferred  \
0  2025-03-10 12:00:00      101  192.168.1.1          LOGIN              4000   
1  2025-03-10 12:05:00      102  192.168.1.2   FAILED_LOGIN               600   
2  2025-03-10 12:10:00      103  192.168.1.3  FILE_DOWNLOAD              5000   
3  2025-03-10 12:15:00      104  192.168.1.4  DATA_TRANSFER             10000   

   failed_logins  successful_logins  label  
0              7                  1      1  
1              2                  1      0  
2              0                  1      0  
3              6                  1      1  

Processed Features:
    user_id  event_type  data_transferred  failed_logins  successful_logins  \
0        0           3              4000              7                  1   
1        1           1               600              2                  1   
2        2           2              5000              0                  1   
3        3           0  

In [30]:
X.head()

,user_id,event_type,data_transferred,failed_logins,successful_logins,failed_login_ratio
0,0,3,4000,7,1,3.5
1,1,1,600,2,1,1.0
2,2,2,5000,0,1,0.0
3,3,0,10000,6,1,3.0


In [32]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from imblearn.over_sampling import SMOTE

#smote = SMOTE(sampling_strategy=0.5, random_state=42)
#X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 4: Model Evaluation
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Step 5: Feature Importance Visualization
feature_importances = pd.Series(clf.feature_importances_, index=X.columns)
#feature_importances.nlargest(10).plot(kind='barh', title="Feature Importance")
#plt.show()


Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



In [34]:
#manual testing with single log
nl = pd.DataFrame({
    'user_id': [105],
    'event_type': [2],
    'data_transferred': [7000],
    'failed_logins': [2],
    'successful_logins': [1],
})
nl['failed_login_ratio'] = nl['failed_logins'] / (nl['successful_logins'] + 1)

prediction=clf.predict(nl)
print("Prediction:", prediction[0])

Prediction: 0
